<a href="https://colab.research.google.com/github/ssnirgudkar/Home-AIML/blob/main/utility_remap_segment_io_instanceids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



After exporting the release file the following has to be done - 
1. Manually download the release file fro each folder (this is needed because the below script uses that version of json 
2. Resize all images files created as part of the export from from 640*512 to 320 * 256
3. Remap the pixel values to category ids - basically chaneg from instance segmentation to semantic. Eg - Water can have a pixel value of 1 in one image while 3 in another image. However category id is constant. Hence we have to update the pixel value of water pixels to the category id of water so that all water pixels have the same value. 
4. Our segmens.io classes are 7. For running against WASR code need to convert them to 3 (sky, water and obstacle). In some images, the instance
id can be greater than the number mentioned below. You will get compiler error if this script comes across such an image. In this case, simply increment the constant by 1 (or just run it using 20 instead of current 11).
5. All output files are stores in the same directory (if on umass lowell machine, have to be uploaded to google drive) 
5. After these steps are done, either the WASR or UNET algorithm can be run 




In [ ]:
def resize_and_save(fileName, new_image_size):
    height, width = new_image_size.split(",")
    img = cv2.imread(fileName, cv2.IMREAD_UNCHANGED)
    dsize = (int(width), int(height))
    resizedImg = cv2.resize(img, dsize)
    tokens = os.path.split(fileName)
    dirs = tokens[:-1]
    rawfileName = tokens[-1][:-4] # last token (-1) is full file name. (-4) is without ".png"
    ext = tokens[-1][-4:]
    outputfilename = rawfileName + "_resized" + ext
    fulloutputfilename = os.path.join(dirs[0], outputfilename)
    cv2.imwrite(fulloutputfilename, resizedImg)

In [ ]:


import os
import sys
import json
import argparse
import cv2

WASR_SKY_ID = 2
WASR_WATER_ID = 1
WASR_OBSTACLE_ID = 0

# These labels should be read from JSON file but for now are hard-coded. Below are category ids
OUR_SKY_ID = 0
OUR_WATER_ID = 1
OUR_STRUCTURE_ID = 2
OUR_OBSTACLE_ID = 3
OUR_LIVING_OBSTACLE_ID = 4
OUR_BACKGROUND_ID = 5
OUR_SELF_ID = 6

TOTAL_INSTANCES = 20

def get_arguments():

    parser = argparse.ArgumentParser(description="JSON reader")
    parser.add_argument("--json-file", type=str, help="Provide full path of JSON file containing instance segmentation info")
    parser.add_argument("--image-file", type=str, 
                        help="Provide full path for a single mask file")
    parser.add_argument("--image-dir", type=str, 
                        help="Provide full path of a directory containing mask files")  
    
    return parser.parse_args()

def readJSONFile(jsonFilePath):
    fileH = open(jsonFilePath)
    data = json.load(fileH)

    #print(data)
    dictImageRemap = {}
    
    for index in data['dataset']['samples']:
        print("record of = {0}".format(index['name']))
        # The instance ids 'id' in JSON file can technically have no upper bound. This is because
        # if there are number of objects in the scene, instance ids will just keep increasing.
        # So the below number needs to be adjusted depending on what you find in JSON file.
        # If JSON file has larger number in its 'id' field than below then adjust the number below
        # to match the high number in JSON file. Specifically it is 'number+1'.
        mapIds = [None] * TOTAL_INSTANCES
        #print(index['name'], index['labels']['ground-truth']['attributes']['annotations'])
        # if images are not labeled then JSON file will contain 
        # ground-truth = null. Handle the case of if index['labels']['ground-truth'] exists
        if index['labels']['ground-truth'] is not None:
            #print("len(index['labels']['ground-truth'])={0}".format(len(index['labels']['ground-truth'])))
            if 'attributes' in index['labels']['ground-truth']:
                if len(index['labels']['ground-truth']['attributes']['annotations']) > 0:
                    #print(i, index['labels']['ground-truth']['attributes']['annotations'][i]['category_id'], index['labels']['ground-truth']['attributes']['annotations'][i]['id'])
                    for jj in range(len(index['labels']['ground-truth']['attributes']['annotations'])):
                        #print("id={0} will be replaced by".format(index['labels']['ground-truth']['attributes']['annotations'][jj]['id']))
                        #print("category_id={0}".format(index['labels']['ground-truth']['attributes']['annotations'][jj]['category_id']))
                        print("index['labels']['ground-truth']['attributes']['annotations'][jj]['id']={0}".format(index['labels']['ground-truth']['attributes']['annotations'][jj]['id']))
                        print("index['labels']['ground-truth']['attributes']['annotations'][jj]['category_id']={0}".format(index['labels']['ground-truth']['attributes']['annotations'][jj]['category_id']))
                        mapIds[index['labels']['ground-truth']['attributes']['annotations'][jj]['id']] = index['labels']['ground-truth']['attributes']['annotations'][jj]['category_id']
                    #print("mapIds={0}".format(mapIds))    
        dictImageRemap[index['name']] = mapIds
        #print("dictImageRemap[index['name']]={0}".format(dictImageRemap[index['name']]))

    fileH.close()
    return dictImageRemap

# This function creates 2 mask image files for a single input mask image.
# First conversion is from segments.ai's instance segmentation map to semantic segmentation map
# Second conversion is from semantic segmentation map to wasr 3 classes map.
# By saving 2 separate mask images, it is easy to carry out experimentation.
# It is expected that the input files are resized appropriately by running imageutilities.py
# script so that its name conforms to the naming convention i.e. the file name contains "_".
def change_one_label_with_another_label(fileOrDirName, records):  

    def change_label(filename, mapRelabel):
        img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        if (img is None):
            print("Image = {0} does not exist.".format(filename))

        img2 = img.copy()
        height,width = img.shape

        dir_filename_pair = os.path.split(filename)
        print("dir_filename_pair={0}".format(dir_filename_pair))
        ext = dir_filename_pair[-1][-4:]
        dirName = dir_filename_pair[:-1]
        print("dir={0}".format(dirName[0]))
        oneDirUp = os.path.abspath(os.path.join(dirName[0], os.pardir))
        print("oneDirUp={0}".format(oneDirUp))
        inputFileName = dir_filename_pair[-1]
        print("inputFileName={0}".format(inputFileName))
        tokens2 = inputFileName.split('_')
        print("tokens2={0}".format(tokens2))
        origImageName = tokens2[0] + ext
        print("origImageName={0}".format(origImageName))
        rawfileName = dir_filename_pair[-1][:-4] # last token (-1) is full file name. (-4) is without ".png"

        
        semantic_segment_dir_name = os.path.join(oneDirUp, 'segment_relabel_semantic')
        print("semantic_segment_dir_name={0}".format(semantic_segment_dir_name))
        semantic_segment_dir = semantic_segment_dir_name
        if not os.path.isdir(semantic_segment_dir_name):
            os.mkdir(semantic_segment_dir_name)
            print("semantic_segment_dir={0}".format(semantic_segment_dir))
        
        outputfilename2 = rawfileName + "_relabeled_semantically" + ext
        fulloutputfilename2 = os.path.join(semantic_segment_dir, outputfilename2)

        print("input file = {0}".format(inputFileName))
        mapOfThisFile = records[origImageName]
        print("mapOfThisFile={0}".format(mapOfThisFile))
        for i in range(height):
            for j in range(width):
                for k in range(len(mapOfThisFile)):
                    if ((img.item(i,j) == k) and (mapOfThisFile[k] is not None)):
                        #print("i={3},j={4},img.item(i,j)={0}, k={1}, mapOfThisFile[k]={2}".format(img.item(i,j), k, mapOfThisFile[k],i,j))
                        img2.itemset((i,j), mapOfThisFile[k])

        cv2.imwrite(fulloutputfilename2, img2)

        relabel2_wasr3_dir_name = os.path.join(oneDirUp, 'segment_relabel2_wasr3')
        relabel2_wasr3_dir = relabel2_wasr3_dir_name
        if not os.path.isdir(relabel2_wasr3_dir_name):
            os.mkdir(relabel2_wasr3_dir_name)

        outputfilename3 = rawfileName + "_relabeled_to3wasrclasses" + ext
        fulloutputfilename3 = os.path.join(relabel2_wasr3_dir, outputfilename3)
        our2WasrMap = [None] * 8
        our2WasrMap[OUR_SKY_ID] = WASR_SKY_ID
        our2WasrMap[OUR_WATER_ID] = WASR_WATER_ID
        our2WasrMap[OUR_STRUCTURE_ID] = OUR_OBSTACLE_ID
        our2WasrMap[OUR_OBSTACLE_ID] = WASR_OBSTACLE_ID
        # Keep the remaining labels intact
        our2WasrMap[OUR_LIVING_OBSTACLE_ID] = our2WasrMap[OUR_BACKGROUND_ID] = our2WasrMap[OUR_SELF_ID] = WASR_OBSTACLE_ID
        img3 = img2.copy()

        for i in range(height):
            for j in range(width):
                for k in range(len(our2WasrMap)):
                    if ((img2.item(i,j) == k) and (our2WasrMap[k] is not None)):
                        #print("img2.item(i,j)={0}, k={1}, our2WasrMap[k]={2}".format(img2.item(i,j), k, our2WasrMap[k]))
                        img3.itemset((i,j), our2WasrMap[k])


        
        cv2.imwrite(fulloutputfilename3, img3)            

    if (os.path.isdir(fileOrDirName)):
        for eachFile in os.listdir(fileOrDirName):
            if (eachFile.endswith(".png") or eachFile.endswith(".jpg")):
                eachFileFullPath = os.path.join(fileOrDirName, eachFile)
                print("eachFile = {0}".format(eachFileFullPath))
                change_label(eachFileFullPath, records)
    else:
        change_label(fileOrDirName, records)    

if __name__ == "__main__":

    args = get_arguments()

    if(args.json_file):
        records = readJSONFile(args.json_file)
        print(records)
        if(args.image_file):
            change_one_label_with_another_label(args.image_file, records)
        elif(args.image_dir):
            change_one_label_with_another_label(args.image_dir, records)

In [ ]:
# this is the shell script to invoke the above 
# This script is a wrapper over python script 'processMaskForSemanticSegmentation'
# It is used to relabel mask images from instance segmentation ids to semantic segmentation ids
# and further to wasr class labels.
# For every mask image, the script creates 2 mask images - one by collapsing instance class ids to
# semantic class ids, second by converting semantic class ids to wasr class ids.
# The first conversion is based on JSON file which must be provided and it is DIFFERENT for DIFFERENT
# directory so care must be taken.
# The semantic id to wasr id remapping is HARD CODED inside processMaskForSemanticSegmentation and it
# should be provided as an argument in future.
INPUTDIR=./segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0
JSONFILE=$INPUTDIR/JSON/IR-2020-10-22-17-33-25-0-V2.0.json
#MASKIMAGEFILE=$INPUTDIR/segmentation_ground_truth_resized/1571161605.647641_label_ground-truth_resized.png
#python processMaskForSemanticSegmentation.py --json-file $JSONFILE --image-file $MASKIMAGEFILE
MASKIMAGEDIR=$INPUTDIR/segmentation_ground_truth_resized
python processMaskForSemanticSegmentation.py --json-file $JSONFILE --image-dir $MASKIMAGEDIR